In [1]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200515142757-0000
KERNEL_ID = 9632965a-2131-4d98-847b-aa6a9678a879
--2020-05-15 14:28:00--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-05-15 14:28:00--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-05-15 14:28:01--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.gi

In [2]:
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [7]:
#as for dataframe df we alraedy crate Temparory view for writting sql-queries on dataframe df
#as------df.createOrReplaceTempView('df')
#here we selecting another column which is continous as SLr-mata-data energy value which is sufficient for implementing linear regression
df_energy=spark.sql("""
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label ,class from df group by class
""")
df_energy.createOrReplaceTempView('df_energy')

In [8]:
df_energy.show()

+------------------+--------------+
|             label|         class|
+------------------+--------------+
| 8959.680239829991| Use_telephone|
| 9737.511232342687| Standup_chair|
| 12542.96539897962|      Eat_meat|
|13225.945637269193|     Getup_bed|
|15003.269043778426|   Drink_glass|
|14454.885091207056|    Pour_water|
|10616.408809008817|     Comb_hair|
|16537.370891408344|          Walk|
|11082.626493751379|  Climb_stairs|
|10261.338314274606| Sitdown_chair|
|6783.4063714331605|   Liedown_bed|
| 7173.493500380411|Descend_stairs|
| 11785.39634462923|   Brush_teeth|
| 6071.460120926432|      Eat_soup|
+------------------+--------------+



In [12]:
#now let join this to the original dataframe df(by innerjoint) :
df_join=spark.sql("""
select * from df inner join df_energy on df.class=df_energy.class
""")


In [13]:
df_join.show()#here you can see that energy values of  several classes are inner jointed to the df by coreesponding class wise where its group by classes

+---+---+---+--------------------+-----------+-----------------+-----------+
|  x|  y|  z|              source|      class|            label|      class|
+---+---+---+--------------------+-----------+-----------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|

In [14]:
from pyspark.ml.regression import LinearRegression 

In [23]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

vectorAssembler=VectorAssembler(inputCols=['x','y','z'],outputCol='features')
normalizer=Normalizer(inputCol='features',outputCol='features_norm',p=1.0)



In [16]:
lr= LinearRegression(maxIter = 10 , regParam=0.3 , elasticNetParam=0.8)

In [25]:
from pyspark.ml import Pipeline
pipeline=Pipeline(stages=[vectorAssembler,normalizer,lr])
model=pipeline.fit(df_join)
prediction=model.transform(df_join)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/opt/ibm/spark/python/pyspark/ml/wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [26]:
prediction.show()

+---+---+---+--------------------+-----------+-----------------+-----------+----------------+--------------------+------------------+
|  x|  y|  z|              source|      class|            label|      class|        features|       features_norm|        prediction|
+---+---+---+--------------------+-----------+-----------------+-----------+----------------+--------------------+------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|12586.729735016828|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,49.0,35.0]|[0.20754716981132...|12586.729735016828|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|12542.703337345756|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|[22.0,52.0,35.0]|[0.20183486238532...|12542.703337345756|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.3963446292

In [27]:
model.stages[2].summary.r2
#evaluating the linear regression model by r2-score

0.03259100556263628

In [ ]:
#as above r2-score is not very much efficient but its worth for the illusration 

In [63]:
#now we illustrating the LogisticRegression

df.show()


+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [64]:
#now splitting the dataset
splits=df.randomSplit([0.8,0.2])
df_train=splits[0]
df_test=splits[1]

In [65]:
df_train.show()#

+---+---+---+--------------------+--------------+
|  x|  y|  z|              source|         class|
+---+---+---+--------------------+--------------+
|  0| 11| 38|Accelerometer-201...| Sitdown_chair|
|  0| 12| 39|Accelerometer-201...| Sitdown_chair|
|  0| 15| 39|Accelerometer-201...|   Brush_teeth|
|  0| 16| 31|Accelerometer-201...|     Getup_bed|
|  0| 17| 36|Accelerometer-201...|   Brush_teeth|
|  0| 23| 36|Accelerometer-201...|   Brush_teeth|
|  0| 24| 35|Accelerometer-201...| Sitdown_chair|
|  0| 25| 30|Accelerometer-201...|   Brush_teeth|
|  0| 26| 15|Accelerometer-201...|  Climb_stairs|
|  0| 26| 42|Accelerometer-201...|   Brush_teeth|
|  0| 27| 31|Accelerometer-201...| Sitdown_chair|
|  0| 27| 33|Accelerometer-201...|     Getup_bed|
|  0| 27| 41|Accelerometer-201...|   Brush_teeth|
|  0| 28| 28|Accelerometer-201...|   Brush_teeth|
|  0| 28| 48|Accelerometer-201...|   Brush_teeth|
|  0| 29| 25|Accelerometer-201...|     Getup_bed|
|  0| 29| 25|Accelerometer-201...|  Climb_stairs|


In [66]:
df_test.show()

+---+---+---+--------------------+--------------+
|  x|  y|  z|              source|         class|
+---+---+---+--------------------+--------------+
|  0| 10| 28|Accelerometer-201...|     Getup_bed|
|  0| 25| 40|Accelerometer-201...|   Brush_teeth|
|  0| 27| 37|Accelerometer-201...|   Brush_teeth|
|  0| 27| 39|Accelerometer-201...|   Brush_teeth|
|  0| 29| 17|Accelerometer-201...|     Getup_bed|
|  0| 29| 38|Accelerometer-201...|   Brush_teeth|
|  0| 29| 43|Accelerometer-201...|   Brush_teeth|
|  0| 30| 36|Accelerometer-201...|  Climb_stairs|
|  0| 30| 43|Accelerometer-201...|   Brush_teeth|
|  0| 31| 17|Accelerometer-201...| Standup_chair|
|  0| 32| 23|Accelerometer-201...|     Getup_bed|
|  0| 32| 33|Accelerometer-201...|          Walk|
|  0| 33| 27|Accelerometer-201...|Descend_stairs|
|  0| 33| 31|Accelerometer-201...|Descend_stairs|
|  0| 33| 35|Accelerometer-201...|          Walk|
|  0| 33| 35|Accelerometer-201...|          Walk|
|  0| 33| 38|Accelerometer-201...|Descend_stairs|


In [68]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler, Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline

stringindexer = StringIndexer(inputCol='class', outputCol='label')

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [69]:
from pyspark.ml.classification import LogisticRegression

In [70]:
lr= LogisticRegression(maxIter=10, regParam=0.3 ,elasticNetParam=0.8)

In [71]:
#now creating pipeline
pipeline=Pipeline(stages=[stringindexer ,vectorAssembler,normalizer,lr])
model=pipeline.fit(df_train)

In [72]:
predictions=model.transform(df_train)

In [73]:
predictions.show()#jst look at it

+---+---+---+--------------------+--------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  x|  y|  z|              source|         class|label|       features|       features_norm|       rawPrediction|         probability|prediction|
+---+---+---+--------------------+--------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  0| 11| 38|Accelerometer-201...| Sitdown_chair|  8.0|[0.0,11.0,38.0]|[0.0,0.2244897959...|[1.25784217625196...|[0.20706476253427...|       0.0|
|  0| 12| 39|Accelerometer-201...| Sitdown_chair|  8.0|[0.0,12.0,39.0]|[0.0,0.2352941176...|[1.25784217625196...|[0.20706476253427...|       0.0|
|  0| 15| 39|Accelerometer-201...|   Brush_teeth|  6.0|[0.0,15.0,39.0]|[0.0,0.2777777777...|[1.25784217625196...|[0.20706476253427...|       0.0|
|  0| 16| 31|Accelerometer-201...|     Getup_bed|  1.0|[0.0,16.0,31.0]|[0.0,0.3404255319...|[1.25784217625196...|[0.20706476

In [74]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [75]:
eval=MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')#creating the instance for multiclassevaluator

In [76]:
eval.evaluate(predictions)#for train data

0.20706459974359406

In [77]:
model=pipeline.fit(df_test)

In [79]:
predictions= model.transform(df_test)

In [81]:
eval.evaluate(predictions)#for the test data

0.20475342450410491